In [44]:
# -*- coding: utf-8 -*-
import dataiku
import pandas as pd, numpy as np
from dataiku import pandasutils as pdu
from bs4 import BeautifulSoup
import requests
import time
from googletrans import Translator

In [10]:
def connect_url(target_url):
    """
    対象のURLにアクセスする関数
    アクセスできない等のエラーが発生したら例外を投げる
    """
    # 接続確立の待機時間、応答待機時間を10秒とし、それぞれの値を超えた場合は例外が発生（ConnectTimeout）
    data = requests.get(target_url, timeout=10)
    data.encoding = data.apparent_encoding
    # アクセス過多を避けるため、2秒スリープ
    time.sleep(2)

    # レスポンスのステータスコードが正常(200番台)以外の場合は、例外を発生させる(HTTPError)
    if data.status_code == requests.codes.ok:
        return data
    else:
        data.raise_for_status()

In [11]:
url = 'https://www.tripadvisor.com/Restaurant_Review-g187147-d10085290-Reviews-Kodawari_Ramen_Yokocho-Paris_Ile_de_France.html'
response = connect_url(url)


In [12]:
soup = BeautifulSoup(response.text, 'lxml')

In [35]:
reviews = []
review_container = soup.find_all(class_='review-container')
for i in range(len(review_container)):
    review = review_container[i].find_all("p", class_='partial_entry')[0].text
    reviews.append(review)

'We arrived for the opening time of 12h00, waited for 25min in the queue, got in to then realised they made a mistake and had us go out again. Another waiting time over 25min to finally get in. The food was not really exceptional. It...is supposed to be made of french ingredient (free range chicken, i dt think this came accross in the taste of the food). It is fairly pricey for what it is.The only positive aspect i could find was the atmosphere, trying to give some authenthic experience.More'

In [0]:
translator = Translator()


In [0]:
# Compute recipe outputs
# TODO: Write here your actual code that computes the outputs
# NB: DSS supports several kinds of APIs for reading and writing data. Please see doc.

your_trip_advisor_df = ... # Compute a Pandas dataframe to write into your_trip_advisor


# Write recipe outputs
your_trip_advisor = dataiku.Dataset("your_trip_advisor")
your_trip_advisor.write_with_schema(your_trip_advisor_df)